In [1]:
""" Supply data is available in m3second and millionm3. Convert to flux.
-------------------------------------------------------------------------------
PCRGLOBWB Data for supply is provided in fluxes with units m3second. A 
previous script converted the supply data to volumes (millionm3)
(implicit per pixel/per time step); converting to fluxes (m) and storing to 
earth engine asset. 

Author: Rutger Hofste
Date: 20180501
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name
    EE_VERSION (integer) : Earth Engine path version.
    INPUT_VERSION (integer) : Input version.
    OUTPUT_VERSION (integer) : Output version.
    TESTING (boolean) : Testing.

Returns:

"""

SCRIPT_NAME = "Y2018M05D01_RH_Supply_Fluxes_5min_EE_V01"
EE_VERSION = 9
INPUT_VERSION = 1
OUTPUT_VERSION = 1 
TESTING = 0

ee_path = "projects/WRI-Aquaduct/PCRGlobWB20V{:02.0f}".format(EE_VERSION)

print("Input ee: " + ee_path +
      "\nOutput ee: " + ee_path )


Input ee: projects/WRI-Aquaduct/PCRGlobWB20V09
Output ee: projects/WRI-Aquaduct/PCRGlobWB20V09


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M05D01 UTC 10:10


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import subprocess
import pandas as pd
import ee
import re
from retrying import retry
from datetime import timedelta
import aqueduct3

ee.Initialize()

In [4]:
def update_property_script_used(image):
    image = image.set("script_used",SCRIPT_NAME)
    return image

def update_property_output_version(image):
    image = image.set("output_version",OUTPUT_VERSION)
    return image

def ic_volume_to_flux_5min_millionm3_m2(ic_input_asset_id,output_version):
    """ Convert an imagecollection from volume to flux.
    -------------------------------------------------------------------------------
    The result is stored in an imagecollection with the same name as the input
    imagecollection but with 'm' instead of 'millionm3'.
    
    Input ic:
    global_historical_PDomWW_year_millionm3_5min_1960_2014
    
    Output ic:    
    global_historical_PDomWW_year_m_5min_1960_2014
    
    Args:
        ic_input_asset_id (string) : asset id of input imagecollection.

    """
    start_time = time.time()
    df_errors = pd.DataFrame()
    
    df = aqueduct3.earthengine.get_df_from_ic(ic_input_asset_id)
    df = aqueduct3.earthengine.add_export_parameters_for_export(df,old_unit,new_unit,output_version)
    
    # Creating ImageCollection(s)
    output_ic_asset_ids = list(df["output_ic_asset_id"].unique())
    for output_ic_asset_id in output_ic_asset_ids:
        command, result = aqueduct3.earthengine.create_imageCollection(output_ic_asset_id)
        print(command,result)
    
    
    # Bacth Converting and uploading.     
    for index, row in df.iterrows():
        elapsed_time = time.time() - start_time 
        print("Index: {:04.0f} Elapsed: {}".format(index, timedelta(seconds=elapsed_time)))
    
        description = row["description"]
        output_image_asset_id = row["output_image_asset_id"]
    
    
        if aqueduct3.earthengine.asset_exists(output_image_asset_id):
            print("Asset exists, skipping: {}".format(output_image_asset_id))
        else:
            i_volume_millionm3_5min = ee.Image(row["input_image_asset_id"])
            i_flux_m_5min = aqueduct3.earthengine.volume_to_flux_5min_millionm3_m2(i_volume_millionm3_5min)
            i_flux_m_5min = update_property_script_used(i_flux_m_5min)
            i_flux_m_5min = update_property_output_version(i_flux_m_5min)
            

            aqueduct3.earthengine.export_image_global_5min(i_flux_m_5min,description,output_image_asset_id)
            print(output_image_asset_id)    
    
    return df


In [5]:
old_unit = "millionm3"
new_unit = "m"

temporal_resolutions = ["year","month"]
indicators = ["riverdischarge"]

if TESTING:
    temporal_resolutions = ["year"]
    
df = pd.DataFrame()
for indicator in indicators:
    for temporal_resolution in temporal_resolutions:
        print(indicator,temporal_resolution)
        ic_input_file_name = "global_historical_{}_{}_millionm3_5min_1960_2014".format(indicator,temporal_resolution)
        ic_input_asset_id = "{}/{}".format(ee_path,ic_input_file_name)
        df = ic_volume_to_flux_5min_millionm3_m2(ic_input_asset_id,OUTPUT_VERSION)    
    

riverdischarge year
earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014 b'Asset projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014 already exists\n'
Index: 0000 Elapsed: 0:00:03.755448
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I000Y1960M01
Index: 0001 Elapsed: 0:00:08.751367
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I001Y1961M01
Index: 0002 Elapsed: 0:00:10.335410
Asset exists, skipping: projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I002Y1962M01
Index: 0003 Elapsed: 0:00:12.384864
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarg

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I041Y2001M01
Index: 0042 Elapsed: 0:02:38.378719
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I042Y2002M01
Index: 0043 Elapsed: 0:02:41.667590
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I043Y2003M01
Index: 0044 Elapsed: 0:02:45.390244
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I044Y2004M01
Index: 0045 Elapsed: 0:02:49.109019
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_year_m_5min_1960_2014/global_historical_riverdischarge_year_m_5min_1960_2014_I045Y2005M01
Index: 0046 Elapsed: 0:02:52.341409
projects/WRI-Aquaduc

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I027Y1962M04
Index: 0028 Elapsed: 0:01:39.757834
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I028Y1962M05
Index: 0029 Elapsed: 0:01:43.407639
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I029Y1962M06
Index: 0030 Elapsed: 0:01:47.129651
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I030Y1962M07
Index: 0031 Elapsed: 0:01:49.913352
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I031Y1962M08
Index: 0032 Elapsed: 0:01:53.122121
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I069Y1965M10
Index: 0070 Elapsed: 0:04:09.331259
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I070Y1965M11
Index: 0071 Elapsed: 0:04:13.581599
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I071Y1965M12
Index: 0072 Elapsed: 0:04:17.292626
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I072Y1966M01
Index: 0073 Elapsed: 0:04:20.481891
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I073Y1966M02
Index: 0074 Elapsed: 0:04:24.178053
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I111Y1969M04
Index: 0112 Elapsed: 0:06:45.747635
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I112Y1969M05
Index: 0113 Elapsed: 0:06:49.398961
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I113Y1969M06
Index: 0114 Elapsed: 0:06:53.070129
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I114Y1969M07
Index: 0115 Elapsed: 0:06:55.999428
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I115Y1969M08
Index: 0116 Elapsed: 0:06:59.228870
projects/W

...

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I489Y2000M10
Index: 0490 Elapsed: 0:29:17.027923
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I490Y2000M11
Index: 0491 Elapsed: 0:29:20.689167
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I491Y2000M12
Index: 0492 Elapsed: 0:29:23.963867
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I492Y2001M01
Index: 0493 Elapsed: 0:29:27.139804
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I493Y2001M02
Index: 0494 Elapsed: 0:29:30.770101
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I531Y2004M04
Index: 0532 Elapsed: 0:31:48.689974
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I532Y2004M05
Index: 0533 Elapsed: 0:31:51.870133
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I533Y2004M06
Index: 0534 Elapsed: 0:31:55.034839
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I534Y2004M07
Index: 0535 Elapsed: 0:31:58.229060
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I535Y2004M08
Index: 0536 Elapsed: 0:32:01.884417
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I573Y2007M10
Index: 0574 Elapsed: 0:34:13.344625
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I574Y2007M11
Index: 0575 Elapsed: 0:34:17.003144
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I575Y2007M12
Index: 0576 Elapsed: 0:34:20.645316
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I576Y2008M01
Index: 0577 Elapsed: 0:34:23.819682
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I577Y2008M02
Index: 0578 Elapsed: 0:34:27.059595
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I615Y2011M04
Index: 0616 Elapsed: 0:36:47.999986
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I616Y2011M05
Index: 0617 Elapsed: 0:36:51.698381
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I617Y2011M06
Index: 0618 Elapsed: 0:36:54.966495
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I618Y2011M07
Index: 0619 Elapsed: 0:36:58.549725
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I619Y2011M08
Index: 0620 Elapsed: 0:37:02.214086
projects/W

projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I657Y2014M10
Index: 0658 Elapsed: 0:39:18.646431
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I658Y2014M11
Index: 0659 Elapsed: 0:39:22.232641
projects/WRI-Aquaduct/PCRGlobWB20V09/global_historical_riverdischarge_month_m_5min_1960_2014/global_historical_riverdischarge_month_m_5min_1960_2014_I659Y2014M12


In [6]:
df.head()

,input_image_asset_id,input_ic_asset_id,output_ic_asset_id,output_image_asset_id,description
0,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_riverdischarge_month_m_5min_...
1,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_riverdischarge_month_m_5min_...
2,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_riverdischarge_month_m_5min_...
3,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_riverdischarge_month_m_5min_...
4,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,projects/WRI-Aquaduct/PCRGlobWB20V09/global_hi...,global_historical_riverdischarge_month_m_5min_...


In [7]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:42:49.746473


Previous runs:  
0:42:49.746473